In [1]:
import math
import pickle
from shapely.geometry import Point
import geopandas as gpd


def getPathLength(lat1,lng1,lat2,lng2):
    '''calculates the distance between two lat, long coordinate pairs'''
    R = 6371000 # radius of earth in m
    lat1rads = math.radians(lat1)
    lat2rads = math.radians(lat2)
    deltaLat = math.radians((lat2-lat1))
    deltaLng = math.radians((lng2-lng1))
    a = math.sin(deltaLat/2) * math.sin(deltaLat/2) + math.cos(lat1rads) * math.cos(lat2rads) * math.sin(deltaLng/2) * math.sin(deltaLng/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d

def getDestinationLatLong(lat,lng,azimuth,distance):
    '''returns the lat an long of destination point 
    given the start lat, long, aziuth, and distance'''
    R = 6378.1 #Radius of the Earth in km
    brng = math.radians(azimuth) #Bearing is degrees converted to radians.
    d = distance/1000 #Distance m converted to km
    lat1 = math.radians(lat) #Current dd lat point converted to radians
    lon1 = math.radians(lng) #Current dd long point converted to radians
    lat2 = math.asin(math.sin(lat1) * math.cos(d/R) + math.cos(lat1)* math.sin(d/R)* math.cos(brng))
    lon2 = lon1 + math.atan2(math.sin(brng) * math.sin(d/R)* math.cos(lat1), math.cos(d/R)- math.sin(lat1)* math.sin(lat2))
    #convert back to degrees
    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)
    return[lat2, lon2]

def calculateBearing(lat1,lng1,lat2,lng2):
    '''calculates the azimuth in degrees from start point to end point'''
    startLat = math.radians(lat1)
    startLong = math.radians(lng1)
    endLat = math.radians(lat2)
    endLong = math.radians(lng2)
    dLong = endLong - startLong
    dPhi = math.log(math.tan(endLat/2.0+math.pi/4.0)/math.tan(startLat/2.0+math.pi/4.0))
    if abs(dLong) > math.pi:
         if dLong > 0.0:
             dLong = -(2.0 * math.pi - dLong)
         else:
             dLong = (2.0 * math.pi + dLong)
    bearing = (math.degrees(math.atan2(dLong, dPhi)) + 360.0) % 360.0;
    return bearing

def coords_src_dest(interval,azimuth,lat1,lng1,lat2,lng2):
    '''returns every coordinate pair inbetween two coordinate 
    pairs given the desired interval'''

    d = getPathLength(lat1,lng1,lat2,lng2)
    remainder, dist = math.modf((d / interval))
    counter = float(interval)
    coords = []
    coords.append([lat1,lng1])
    for distance in range(0,int(dist)):
        coord = getDestinationLatLong(lat1,lng1,azimuth,counter)
        counter = counter + float(interval)
        coords.append(coord)
    coords.append([lat2,lng2])
    return coords

In [2]:
if __name__ == "__main__":
    #point interval in meters
    interval = 2.0
    #direction of line in degrees
    #start point
    lat1 = 12.794702
    lng1 = 77.624046
    	
    #end point
    lat2 = 12.863790
    lng2 = 77.616608
    azimuth = calculateBearing(lat1,lng1,lat2,lng2)
    print (azimuth)
    coords = coords_src_dest(interval,azimuth,lat1,lng1,lat2,lng2)
    for lat,long in coords:
        print(lat, long)
        break
    # print(getPathLength())

354.0074833512549
12.794702 77.624046


In [8]:
def locs_to_avgspeed(lat1, lng1, lat2, lng2, model):
    interval = 10
    azimuth = calculateBearing(lat1, lng1, lat2, lng2)
    coords =  coords_src_dest(interval, azimuth, lat1, lng1, lat2, lng2)
    speeds = float()
    for lat, lng in coords:
        speeds = speeds + model.predict([[math.radians(lat),math.radians(lng)]])
    return float(speeds / len(coords))
model = pickle.load(open('21nbrs(5.19,67.91,8.24).sav', 'rb'))
# model = pickle.load(open('21nbrs.sav', 'rb'))
lat1 = 12.918303	
lng1 = 77.589500
lat2 = 12.923568	
lng2 = 77.655655
			
speed = locs_to_avgspeed(lat1, lng1, lat2, lng2, model)
distance = getPathLength(lat1, lng1, lat2, lng2) / 1000
print(distance * 60 / speed)
print(distance)


29.668901740112897
7.19370251952082


In [4]:
'''
s = d / t => t = d / s  
km / km h-1 => h * 60 => minutes 
'''

'\ns = d / t => t = d / s  \nkm / km h-1 => h * 60 => minutes \n'